In [1]:
import pandas as pd
import numpy as np
import re

# Load CSVs
food_item = pd.read_csv("food_item.csv")
ingredient = pd.read_csv("ingredient.csv")
recipe = pd.read_csv("recipe.csv")
stock = pd.read_csv("stock.csv")

# Load text files
with open("names.txt") as f:
    names = [line.strip() for line in f]

with open("passwords.txt") as f:
    passwords = [line.strip() for line in f]

with open("cards.txt") as f:
    cards = [line.strip() for line in f]

# Load wordlists (saved as RData originally; here assume CSV/JSON alternative or pickle)
# Example: wordlists = pd.read_pickle("wordlists.pkl")
# We'll mock it here:
wordlists = {
    "Ranges": ["abc", "fff", "def"],
    "Backrefs": ["abcabc", "123123", "xyz"],
    "Prime": ["2", "3", "5", "11", "22"]
}

In [8]:
# --- helper functions to find relevant columns dynamically ---
def find_col(df, keyword):
    """Return first column in df containing keyword (case-insensitive)."""
    matches = [c for c in df.columns if keyword.lower() in c.lower()]
    return matches[0] if matches else None

weight_col = find_col(ingredient, "weight")
calories_col = find_col(food_item, "calorie")
price_col = find_col(stock, "price")

print("Detected cols:", {"weight": weight_col, "calories": calories_col, "price": price_col})


# --- Q1a: Calories in Turkey Burger grouped by type ---
turkey_burger = (
    ingredient.merge(food_item, left_on="food_item", right_on="item", how="right")
    .query('recipe == "Turkey Burger"')
    .groupby("type", as_index=False)[calories_col]
    .sum()
)
print("Q1a:", turkey_burger, "\n")


# --- Q1b: Recipes with both Bread and Tomato ---
bread_tomato = (
    ingredient.loc[ingredient["food_item"].str.contains("Bread|Tomato", regex=True, na=False)]
    .groupby("recipe")
    .filter(lambda g: len(g) > 1)
    .groupby("recipe", as_index=False)[weight_col]
    .sum()
    .rename(columns={weight_col: "oz"})
)
print("Q1b:", bread_tomato, "\n")


# --- Q1c: Total calories for Beef vs Turkey Burger ---
beef_turkey_burger = (
    ingredient.loc[ingredient["recipe"].isin(["Beef Burger", "Turkey Burger"])]
    .merge(food_item, left_on="food_item", right_on="item")
    .assign(tot=lambda df: df[weight_col] * df[calories_col])
    .groupby("recipe", as_index=False)["tot"]
    .sum()
)
print("Q1c:", beef_turkey_burger, "\n")


# --- Q1d: Cheapest vegetable price at W-Mart or Food warehouse ---
price = (
    stock.query('shop in ["W-Mart", "Food warehouse"]')
    .merge(food_item, left_on="food_item", right_on="item")
    .query('type == "Vegetables"')
    .groupby("food_item", as_index=False)[price_col]
    .min()
    .rename(columns={price_col: "min_price"})
    .merge(stock, left_on=["food_item", "min_price"], right_on=["food_item", price_col])
)
print("Q1d:", price, "\n")


# --- Q1e: Wheat products by total calories ---
wheat = (
    ingredient.merge(food_item, left_on="food_item", right_on="item", how="right")
    .query('type == "Wheat product"')
    .groupby("recipe", as_index=False)[calories_col]
    .sum()
    .rename(columns={calories_col: "total_calories"})
    .sort_values("total_calories", ascending=False)
)
print("Q1e:", wheat, "\n")

Detected cols: {'weight': 'weight (oz)', 'calories': 'calories', 'price': 'price (US dollars per lb)'}
Q1a:             type  calories
0           Meat      15.0
1          Sauce      35.0
2     Vegetables       3.5
3  Wheat product      25.0 

Q1b:           recipe  oz
0    Beef Burger  16
1       Sandwich  17
2  Turkey Burger  14 

Q1c:           recipe     tot
0    Beef Burger  1317.0
1  Turkey Burger   777.0 

Q1d:   food_item  min_price            shop  price (US dollars per lb)
0   Cabbage        1.8  Food warehouse                        1.8
1    Onions        1.0          W-Mart                        1.0
2    Onions        1.0       Coco Mart                        1.0
3   Spinach        2.5  Food warehouse                        2.5
4    Tomato        3.0          W-Mart                        3.0 

Q1e:                     recipe  total_calories
0              Beef Burger            25.0
1                 Sandwich            25.0
3            Turkey Burger            25.0
2 

In [5]:
from homework3 import q1_a, q1_b, q1_c, q2_a, q2_b, q3_a, q3_b, q4_a, q4_b, q4_c

# 2.1a
q2_1a = [n for n in names if re.search(q1_a, n)]
print("Q2.1a:", q2_1a)

# 2.1b
q2_1b = [n for n in names if re.search(q1_b, n)]
print("Q2.1b:", q2_1b)

# 2.1c
q2_1c = [n for n in names if re.search(q1_c, n)]
print("Q2.1c:", q2_1c)

# 2.2a
q2_2a = [c for c in cards if re.search(q2_a, c)]
print("Q2.2a:", q2_2a)

# 2.2b
q2_2b = [c for c in cards if re.search(q2_b, c)]
print("Q2.2b:", q2_2b)

# 2.3a
q2_3a = [p for p in passwords if re.search(q3_a, p)]
print("Q2.3a:", q2_3a)

# 2.3b
q2_3b = [p for p in passwords if re.search(q3_b, p)]
print("Q2.3b:", q2_3b)

# 2.4a
ranges = wordlists["Ranges"]
q2_4a = [bool(re.search(q4_a, r)) for r in ranges]
print("Q2.4a:", q2_4a)

# 2.4b
backrefs = wordlists["Backrefs"]
q2_4b = [bool(re.search(q4_b, b)) for b in backrefs]
print("Q2.4b:", q2_4b)

# 2.4c
primes = wordlists["Prime"]
q2_4c = [bool(re.search(q4_c, p)) for p in primes]
print("Q2.4c:", q2_4c)

Q2.1a: ['Abc One23', 'myUsernameis210', '123fionaFog', 'Red chihuahua5', '1', '0maha p0shy']
Q2.1b: ['asml', 'john']
Q2.1c: ['Zoran D Wang', 'Edward Lazowska', 'Ajay Ann Bryan']
Q2.2a: ['5123456789101112', '5234 4567 8910 1112', '5192 9295 91828818']
Q2.2b: ['4789 0123 8910 1112', '4444 9321 1230 3']
Q2.3a: ['Strings78', 'appleO07', '1brownie', 'glhf1234']
Q2.3b: ['Strings78', 'appleO07']
Q2.4a: [True, True, True]
Q2.4b: [True, True, False]
Q2.4c: [True, True, True, False, False]
